In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn.metrics.pairwise import cosine_similarity

from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# load model
model = tf.keras.models.load_model('/content/gdrive/My Drive/data/stereo_matching/model_BC_self_learning.h5')
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 2)]   0         
_________________________________________________________________
sequential (Sequential)      (None, 15, 15, 2)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 13, 13, 32)        608       
_________________________________________________________________
batch_normalization (BatchNo (None, 13, 13, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 11, 11, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 32)          9248  

In [ ]:
def compute_forward(data):
  for layer in model.layers[2:]:
    data = layer(data)
  return data

In [ ]:
left  = cv2.imread("/content/gdrive/My Drive/data/stereo_matching/Aloe_left_1.png", 0)
right = cv2.imread("/content/gdrive/My Drive/data/stereo_matching/Aloe_right_2.png", 0)

#z-score
mean = np.mean(left)
std  = np.std(left)
left = (left - mean)/(std+1e-7)

mean = np.mean(right)
std  = np.std(right)
right = (right - mean)/(std+1e-7)

disparity_range = 64  # D
H, W = right.shape

print(left.shape)
print(right.shape)
print(H, W)

(370, 427)
(370, 427)
370 427


In [ ]:
cost_volume = np.zeros((356, 413, disparity_range))

for loc in range(disparity_range):
    x_off = -loc
    l = left[:, max(0, -x_off):W]
    r = right[:, 0:min(W, W + x_off)]
    
    lr = tf.stack([l,r], axis=2)
    lr = tf.expand_dims(lr, axis=0)
    
    similarity_map = compute_forward(lr)
    cost_volume[:, loc:413, loc] = similarity_map[:, :]

In [ ]:
from PIL import Image

pred = tf.argmax(cost_volume, axis=2)
pred = pred.numpy() * 3.0

# save
pred = pred.astype(np.uint8)
im = Image.fromarray(pred)
im.save('/content/gdrive/My Drive/data/stereo_matching/disparity_self_learning.png')